# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")
do_cat = False
do_cross_match = False
do_src_prep = True
do_variable_addition = False

srcs_ids = []
srcs_ids = [8215 ,32448 ,54184 ,98746 ,151796 ,166179 ,172775 ,193067 ,199832 ,259271 ,265150 ,297278 ,305192 ,388172 ,432606 ,472623 ,535864 ,541266 ,625693 ,1521738 ,1551422 ,2136829 ,] # WD ALL_10-800-LOOSE

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname)
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad")
    rc.cross_match_cds()

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0

    # Add object groups
    add_ogrp(rc,"tt_simbad")
    add_ogrp(rc,"tt_otypes")
    add_ogrp(rc,"tt_gaiadr3",provenance="GAIA")

#Write out all changes
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-10-14 16:14:41.600 | DEBUG    | vasca.tables:load_from_fits:328 - Loading file with name './vasca_pipeline/ALL_10-800_LOOSE/region_ALL_10-800_LOOSE_cat.fits'
2023-10-14 16:14:41.606 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_fields'
2023-10-14 16:14:41.617 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_filters'
2023-10-14 16:14:41.622 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_visits'
2023-10-14 16:14:41.628 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_sources'
2023-10-14 16:14:41.645 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_detections'
2023-10-14 16:14:41.671 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_detections'
2023-10-14 16:14:41.734 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_src_id_map'
2023-10-14 16:14:41.740 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_simbad'
2023-10-14 16:14:41.769 | DEBUG    | vasc

Nr of spectra found 1


2023-10-14 16:14:51.574 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-14 16:14:51.580 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-14 16:14:51.590 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-14 16:14:51.594 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-14 16:14:51.603 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-14 16:14:51.609 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'
2023-10-14 16:14:51.617 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_spectrum'
2023-10-14 16:14:51.768 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 32448
2023-10-14 16:14:51.771 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-14 16:14:51.789 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-14 16:14:51

Nr of spectra found 2


2023-10-14 16:15:13.329 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-14 16:15:13.335 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-14 16:15:13.345 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-14 16:15:13.349 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-14 16:15:13.357 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-14 16:15:13.363 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'
2023-10-14 16:15:13.371 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_spectrum'
2023-10-14 16:15:13.477 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 54184
2023-10-14 16:15:13.479 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-14 16:15:13.496 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-14 16:15:13

Nr of spectra found 1


2023-10-14 16:16:06.941 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_visits'
2023-10-14 16:16:06.946 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2023-10-14 16:16:06.957 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2023-10-14 16:16:06.961 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_source_lc'
2023-10-14 16:16:06.971 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_sed'
2023-10-14 16:16:06.976 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_gphoton_lc'
2023-10-14 16:16:06.985 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_spectrum'
2023-10-14 16:16:07.103 | DEBUG    | vasca.region:get_src_from_id:337 - Getting Source object for rg_src_id: 193067
2023-10-14 16:16:07.106 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_sources'
2023-10-14 16:16:07.124 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-14 16:16:0

Nr of spectra found 2


2023-10-14 16:18:00.941 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_detections'
2023-10-14 16:18:00.949 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_simbad'
2023-10-14 16:18:00.963 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_gaiadr3'
2023-10-14 16:18:00.976 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_visits'
2023-10-14 16:18:00.980 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_fields'
2023-10-14 16:18:00.980 | DEBUG    | vasca.tables:add_table:157 - Adding table 'tt_filters'
2023-10-14 16:18:00.981 | DEBUG    | vasca.tables:get_light_curve:568 - Getting light curve for fd_src_ids 'None' and rg_src_ids '1551422'
2023-10-14 16:18:00.981 | DEBUG    | vasca.tables:get_light_curve:589 - Getting lightcurve for rg_src_id: [1551422]
2023-10-14 16:18:00.992 | DEBUG    | vasca.tables:table_from_template:121 - Created new table from template 'base_field:tt_source_lc'.
2023-10-14 16:18:00.992 | DEBUG    | vasca.tables:add_tabl

Done
